#ASSIGNMENT 2 - AUTO ML
 
**Author: Jatin Madan**

**NUID: 002727159**

Implementing AutoML: Automated machine learning, often known as automated ML or AutoML, is the process of automating the laborious, iterative activities associated with developing a machine learning model. It enables model quality to be maintained while ML models are built at high scale, efficiency, and productivity by data scientists, analysts, and developers.


H2O is a distributed in-memory machine learning platform with linear scalability that can be used to implement AutoML. The most popular statistical and machine learning algorithms, such as deep learning, generalized linear models, and gradient boosted machines, are supported by H2O.


#PROBLEM STATEMENT: 
Using H2O AutoML to train a model that will help us predict whether the customers request for credit card will get approved or not. The dataset we will be working with contains information on various background information of the customers. The main aim of this project is to predict the customers request for credit card will get approved or not.

###INSTALLING H2O

In [1]:
! pip install h2o

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 177.4 MB 40 kB/s 
  Created wheel for h2o: filename=h2o-3.38.0.2-py2.py3-none-any.whl size=177521195 sha256=600ddad463e89d355bda8037b3e33d6fe5002cf9088eab71ca3314e8d00d5a4b
  Stored in directory: /root/.cache/pip/wheels/e4/ef/ab/a9b2e452e18b3dfea0b6114bc57c3b9e8b0e464eb2d03230e1
Successfully built h2o


###IMPORTING LIBRARIES

To use H2O in Python, we first initialize a connection between our Python and an H2O local server. 

Connecting to cluster

In [2]:
# Load the H2O library and start up the H2O cluter locally on your machine
import h2o
import pandas as pd
from h2o.automl import H2OAutoML
import re
import matplotlib.pylab as plt
import numpy as np
# Number of threads, nthreads = -1, means use all cores on your machine
# max_mem_size is the maximum memory (in GB) to allocate to H2O
h2o.init(nthreads = -1, max_mem_size = 8)

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.16" 2022-07-19; OpenJDK Runtime Environment (build 11.0.16+8-post-Ubuntu-0ubuntu118.04); OpenJDK 64-Bit Server VM (build 11.0.16+8-post-Ubuntu-0ubuntu118.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.7/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp17l8nf3l
  JVM stdout: /tmp/tmp17l8nf3l/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp17l8nf3l/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,05 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.2
H2O_cluster_version_age:,11 days
H2O_cluster_name:,H2O_from_python_unknownUser_x3a1e8
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,8 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


**About the Dataset: Credit Card classification**

The bank manager has noticed that more and more customers are applying for their credit card services. They would certainly like it if someone could foresee who can more likely to get approve the credit card request knowing their background details such as Years of Employment, Type of Income, Family Status, etc.

Dataset consists of 9709 customers and there are nearly 20 features.

###LOADING DATASET

In [3]:
# loading datset
path = "/content/clean_data.csv"
data_path = pd.read_csv("/content/clean_data.csv")

In [4]:
# creating a copy of the dataset
df_copy=data_path.copy()

In [6]:
# Replacing the Target binary values into String values to perform Binomial classification
df_copy['Target'] = df_copy['Target'].replace(1, "Y")
df_copy['Target'] = df_copy['Target'].replace(0, "N")

In [7]:
data_path.head(5)

,ID,Gender,Own_car,Own_property,Work_phone,Phone,Email,Unemployed,Num_children,Num_family,Account_length,Total_income,Age,Years_employed,Income_type,Education_type,Family_status,Housing_type,Occupation_type,Target
0,5008804,1,1,1,1,0,0,0,0,2,15,427500.0,32.868574,12.435574,Working,Higher education,Civil marriage,Rented apartment,Other,1
1,5008806,1,1,1,0,0,0,0,0,2,29,112500.0,58.793815,3.104787,Working,Secondary / secondary special,Married,House / apartment,Security staff,0
2,5008808,0,0,1,0,1,1,0,0,1,4,270000.0,52.321403,8.353354,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,Sales staff,0
3,5008812,0,0,1,0,0,0,1,0,1,20,283500.0,61.504343,0.000000,Pensioner,Higher education,Separated,House / apartment,Other,0
4,5008815,1,1,1,1,1,1,0,0,2,5,270000.0,46.193967,2.105450,Working,Higher education,Married,House / apartment,Accountants,0


In [8]:
#Loading the dataset into H2O
data = h2o.H2OFrame(data_path)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [9]:
# description of the dataset
data.describe()

Rows:9709
Cols:20

,ID,Gender,Own_car,Own_property,Work_phone,Phone,Email,Unemployed,Num_children,Num_family,Account_length,Total_income,Age,Years_employed,Income_type,Education_type,Family_status,Housing_type,Occupation_type,Target
type,int,int,int,int,int,int,int,int,int,int,int,real,real,real,enum,enum,enum,enum,enum,int
mins,5008804.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,27000.0,20.504185575336933,0.0,,,,,,0.0
mean,5076104.679060664,0.3487485837882377,0.36770007209805333,0.671541868369554,0.21742712946750437,0.2876712328767123,0.08754763621382222,0.1746832835513441,0.42280358430322246,2.1826140694201257,27.270058708414904,181228.19456174638,43.784093083598485,5.664730161452851,,,,,,0.132145432073334
maxs,5150479.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,19.0,20.0,60.0,1575000.0,68.86383703977495,43.02073280081042,,,,,,1.0
sigma,40802.696052772466,0.4765987878099639,0.4822039797226454,0.4696765995682179,0.4125167873991112,0.45270034532244624,0.2826504487639636,0.3797155310732071,0.7670189141552809,0.9329181887062356,16.648056893148983,99277.30509737751,11.625767724979015,6.3422412768887755,,,,,,0.3386662517937518
zeros,0,6323,6139,3189,7598,6916,8859,8013,6819,0,57,0,0,1696,,,,,,8426
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,5008804.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,2.0,15.0,427500.0,32.86857361889703,12.4355736257418,Working,Higher education,Civil marriage,Rented apartment,Other,1.0
1,5008806.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,29.0,112500.0,58.79381506807121,3.104786545924968,Working,Secondary / secondary special,Married,House / apartment,Security staff,0.0
2,5008808.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,4.0,270000.0,52.32140290355038,8.353354278321937,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,Sales staff,0.0


In [10]:
data.types

{'ID': 'int',
 'Gender': 'int',
 'Own_car': 'int',
 'Own_property': 'int',
 'Work_phone': 'int',
 'Phone': 'int',
 'Email': 'int',
 'Unemployed': 'int',
 'Num_children': 'int',
 'Num_family': 'int',
 'Account_length': 'int',
 'Total_income': 'real',
 'Age': 'real',
 'Years_employed': 'real',
 'Income_type': 'enum',
 'Education_type': 'enum',
 'Family_status': 'enum',
 'Housing_type': 'enum',
 'Occupation_type': 'enum',
 'Target': 'int'}

In [11]:
data.head()

ID,Gender,Own_car,Own_property,Work_phone,Phone,Email,Unemployed,Num_children,Num_family,Account_length,Total_income,Age,Years_employed,Income_type,Education_type,Family_status,Housing_type,Occupation_type,Target
5.0088e+06,1,1,1,1,0,0,0,0,2,15,427500,32.8686,12.4356,Working,Higher education,Civil marriage,Rented apartment,Other,1
5.00881e+06,1,1,1,0,0,0,0,0,2,29,112500,58.7938,3.10479,Working,Secondary / secondary special,Married,House / apartment,Security staff,0
5.00881e+06,0,0,1,0,1,1,0,0,1,4,270000,52.3214,8.35335,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,Sales staff,0
5.00881e+06,0,0,1,0,0,0,1,0,1,20,283500,61.5043,0,Pensioner,Higher education,Separated,House / apartment,Other,0
5.00882e+06,1,1,1,1,1,1,0,0,2,5,270000,46.194,2.10545,Working,Higher education,Married,House / apartment,Accountants,0
5.00882e+06,1,1,1,0,0,0,0,0,2,17,135000,48.6745,3.26906,Commercial associate,Secondary / secondary special,Married,House / apartment,Laborers,0
5.00882e+06,0,1,0,0,0,0,0,0,2,25,130500,29.2107,3.01991,Working,Incomplete higher,Married,House / apartment,Accountants,1
5.00883e+06,0,0,1,0,1,0,0,0,2,31,157500,27.4639,4.02199,Working,Secondary / secondary special,Married,House / apartment,Laborers,1
5.00883e+06,0,0,1,0,0,0,0,1,2,44,112500,30.0294,4.43541,Working,Secondary / secondary special,Single / not married,House / apartment,Other,0
5.00884e+06,1,1,1,0,0,0,0,3,5,24,270000,34.7413,3.18419,Working,Secondary / secondary special,Married,House / apartment,Laborers,0


##MULTINOMIAL CLASSIFICATION

In [12]:
# Splitting data into train test and validation set 
data_train,data_test,data_valid = data.split_frame(ratios=[.7, .15])

In [13]:
data_train.head()

ID,Gender,Own_car,Own_property,Work_phone,Phone,Email,Unemployed,Num_children,Num_family,Account_length,Total_income,Age,Years_employed,Income_type,Education_type,Family_status,Housing_type,Occupation_type,Target
5.0088e+06,1,1,1,1,0,0,0,0,2,15,427500,32.8686,12.4356,Working,Higher education,Civil marriage,Rented apartment,Other,1
5.00881e+06,1,1,1,0,0,0,0,0,2,29,112500,58.7938,3.10479,Working,Secondary / secondary special,Married,House / apartment,Security staff,0
5.00881e+06,0,0,1,0,0,0,1,0,1,20,283500,61.5043,0,Pensioner,Higher education,Separated,House / apartment,Other,0
5.00882e+06,1,1,1,1,1,1,0,0,2,5,270000,46.194,2.10545,Working,Higher education,Married,House / apartment,Accountants,0
5.00882e+06,1,1,1,0,0,0,0,0,2,17,135000,48.6745,3.26906,Commercial associate,Secondary / secondary special,Married,House / apartment,Laborers,0
5.00882e+06,0,1,0,0,0,0,0,0,2,25,130500,29.2107,3.01991,Working,Incomplete higher,Married,House / apartment,Accountants,1
5.00883e+06,0,0,1,0,0,0,0,1,2,44,112500,30.0294,4.43541,Working,Secondary / secondary special,Single / not married,House / apartment,Other,0
5.00884e+06,1,1,1,0,0,0,0,3,5,24,270000,34.7413,3.18419,Working,Secondary / secondary special,Married,House / apartment,Laborers,0
5.00884e+06,1,0,1,0,0,0,0,1,3,39,405000,32.4223,5.51962,Commercial associate,Higher education,Married,House / apartment,Managers,0
5.00884e+06,1,1,1,0,1,0,0,0,2,43,112500,56.1326,12.1837,Commercial associate,Secondary / secondary special,Married,House / apartment,Drivers,0


In [14]:
y = "Income_type"
x = data.columns
x.remove(y)
x

['ID',
 'Gender',
 'Own_car',
 'Own_property',
 'Work_phone',
 'Phone',
 'Email',
 'Unemployed',
 'Num_children',
 'Num_family',
 'Account_length',
 'Total_income',
 'Age',
 'Years_employed',
 'Education_type',
 'Family_status',
 'Housing_type',
 'Occupation_type',
 'Target']

In [15]:
aml = H2OAutoML(max_models = 10, seed = 10, verbosity="info", nfolds=0)

In [16]:
aml.train(x = x, y = y, training_frame = data_train, validation_frame=data_valid)

AutoML progress: |
01:25:56.984: Project: AutoML_1_20221108_12556
01:25:56.988: Cross-validation disabled by user: no fold column nor nfolds > 1.
01:25:56.989: Setting stopping tolerance adaptively based on the training frame: 0.012103663970544886
01:25:56.989: Build control seed: 10
01:25:56.990: training frame: Frame key: AutoML_1_20221108_12556_training_py_3_sid_b8a4    cols: 20    rows: 6826  chunks: 1    size: 232690  checksum: -2171122911107645420
01:25:56.990: validation frame: Frame key: py_5_sid_b8a4    cols: 20    rows: 1388  chunks: 1    size: 53196  checksum: -7138771450889962336
01:25:56.991: leaderboard frame: Frame key: py_5_sid_b8a4    cols: 20    rows: 1388  chunks: 1    size: 53196  checksum: -7138771450889962336
01:25:56.991: blending frame: NULL
01:25:56.995: response column: Income_type
01:25:56.997: fold column: null
01:25:56.997: weights column: null
01:25:57.53: Loading execution steps: [{XGBoost : [def_2 (1g, 10w), def_1 (2g, 10w), def_3 (3g, 10w), grid_1 (4g, 

Model Details
=============
H2OXGBoostEstimator : XGBoost
Model Key: XGBoost_1_AutoML_1_20221108_12556


Model Summary: 
    number_of_trees
--  -----------------
    40

ModelMetricsMultinomial: xgboost
** Reported on train data. **

MSE: 0.10283245938655587
RMSE: 0.32067500586505937
LogLoss: 0.34077475078289077
Mean Per-Class Error: 0.33189337158588456
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Commercial associate    Pensioner    State servant    Student    Working    Error      Rate
----------------------  -----------  ---------------  ---------  ---------  ---------  -----------
1333                    0            11               0          321        0.199399   332 / 1,665
10                      1199         1                0          3          0.0115416  14 / 1,213
46                      0            278              0          171        0.438384   217 / 495
0                       0            1                0          1          1          2 / 2
32                      0            3                0          3416       0.010142   35 / 3,451
1421                    1199         294              0          3912       0.0878992  600 / 6,826

Top-5 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.912101
2    0.983006
3    0.998535
4    0.999707
5    1

ModelMetricsMultinomial: xgboost
** Reported on validation data. **

MSE: 0.2554074854619556
RMSE: 0.5053785565909534
LogLoss: 0.758581398878846
Mean Per-Class Error: 0.5640444214876033
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Commercial associate    Pensioner    State servant    Student    Working    Error       Rate
----------------------  -----------  ---------------  ---------  ---------  ----------  -----------
91                      0            8                0          221        0.715625    229 / 320
1                       240          0                0          1          0.00826446  2 / 242
19                      0            10               0          92         0.917355    111 / 121
1                       0            0                0          0          1           1 / 1
111                     0            15               0          578        0.178977    126 / 704
223                     240          33               0          892        0.337896    469 / 1,388

Top-5 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.662104
2    0.914986
3    0.997118
4    0.99928
5    1

Scoring History: 
    timestamp            duration    number_of_trees    training_rmse    training_logloss    training_classification_error    training_auc    training_pr_auc    validation_rmse    validation_logloss    validation_classification_error    validation_auc    validation_pr_auc
--  -------------------  ----------  -----------------  ---------------  ------------------  -------------------------------  --------------  -----------------  -----------------  --------------------  ---------------------------------  ----------------  -------------------
    2022-11-08 01:25:57  0.644 sec   0                  0.8              1.60944             0.494433                         nan             nan                0.8                1.60944               0.492795                           nan               nan
    2022-11-08 01:25:59  2.281 se

In [17]:
lb = aml.leaderboard

In [18]:
lb.head()

model_id,mean_per_class_error,logloss,rmse,mse
XGBoost_1_AutoML_1_20221108_12556,0.564044,0.758581,0.505379,0.255407
GBM_2_AutoML_1_20221108_12556,0.567955,0.727453,0.499127,0.249128
XGBoost_2_AutoML_1_20221108_12556,0.572562,0.752089,0.50543,0.25546
DRF_1_AutoML_1_20221108_12556,0.573321,0.869104,0.498458,0.248461
GBM_1_AutoML_1_20221108_12556,0.574969,0.707133,0.494705,0.244733
GBM_3_AutoML_1_20221108_12556,0.576451,0.729864,0.499304,0.249305
XGBoost_3_AutoML_1_20221108_12556,0.577247,0.721598,0.499392,0.249392
GBM_4_AutoML_1_20221108_12556,0.579514,0.747299,0.50272,0.252727
GLM_1_AutoML_1_20221108_12556,0.581178,0.697024,0.493499,0.243541
XRT_1_AutoML_1_20221108_12556,0.593017,0.898696,0.571117,0.326174


In [48]:
# finding and storing the best model
best_model = h2o.get_model(aml.leaderboard[8,'model_id'])

In [49]:
# printing the best model
best_model.algo

'glm'

In [19]:
data_pred=aml.leader.predict(data_test)

xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%


In [20]:
data_pred.head()

predict,Commercial associate,Pensioner,State servant,Student,Working
Working,0.133264,0.00168862,0.335141,0.000758007,0.529149
Working,0.185471,0.00135208,0.0173504,0.00112729,0.794699
Working,0.152218,0.00186893,0.0172813,0.000743468,0.827888
Working,0.221623,0.00526579,0.304969,0.00132303,0.466819
Working,0.115255,0.00328182,0.0359004,0.00129779,0.844265
State servant,0.0747061,0.00114836,0.57907,0.000859592,0.344216
Commercial associate,0.463777,0.00557823,0.308203,0.00131319,0.221129
Working,0.423059,0.0192118,0.0465733,0.000697778,0.510458
Working,0.15333,0.00173453,0.101951,0.000736172,0.742248
Working,0.113703,0.000906886,0.0666955,0.000675844,0.818019


In [21]:
aml.leader.model_performance(data_test)

ModelMetricsMultinomial: xgboost
** Reported on test data. **

MSE: 0.24585601901703508
RMSE: 0.4958387026211599
LogLoss: 0.7197876721807833
Mean Per-Class Error: 0.3774286911293482
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Commercial associate    Pensioner    State servant    Student    Working    Error     Rate
----------------------  -----------  ---------------  ---------  ---------  --------  -----------
76                      0            8                0          243        0.767584  251 / 327
0                       257          0                0          0          0         0 / 257
19                      0            8                0          79         0.924528  98 / 106
0                       0            0                0          0          nan       0 / 0
140                     0            17               0          648        0.195031  157 / 805
235                     257          33               0          970        0.338462  506 / 1,495

Top-5 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.661539
2    0.934448
3    0.999331
4    1
5    1

##BINARY CLASSIFICATION

In [22]:
data1 = h2o.H2OFrame(df_copy)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [23]:
data1.describe()

Rows:9709
Cols:20

,ID,Gender,Own_car,Own_property,Work_phone,Phone,Email,Unemployed,Num_children,Num_family,Account_length,Total_income,Age,Years_employed,Income_type,Education_type,Family_status,Housing_type,Occupation_type,Target
type,int,int,int,int,int,int,int,int,int,int,int,real,real,real,enum,enum,enum,enum,enum,enum
mins,5008804.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,27000.0,20.504185575336933,0.0,,,,,,
mean,5076104.679060664,0.3487485837882377,0.36770007209805333,0.671541868369554,0.21742712946750437,0.2876712328767123,0.08754763621382222,0.1746832835513441,0.42280358430322246,2.1826140694201257,27.270058708414904,181228.19456174638,43.784093083598485,5.664730161452851,,,,,,
maxs,5150479.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,19.0,20.0,60.0,1575000.0,68.86383703977495,43.02073280081042,,,,,,
sigma,40802.696052772466,0.4765987878099639,0.4822039797226454,0.4696765995682179,0.4125167873991112,0.45270034532244624,0.2826504487639636,0.3797155310732071,0.7670189141552809,0.9329181887062356,16.648056893148983,99277.30509737751,11.625767724979015,6.3422412768887755,,,,,,
zeros,0,6323,6139,3189,7598,6916,8859,8013,6819,0,57,0,0,1696,,,,,,
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,5008804.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,2.0,15.0,427500.0,32.86857361889703,12.4355736257418,Working,Higher education,Civil marriage,Rented apartment,Other,Y
1,5008806.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,29.0,112500.0,58.79381506807121,3.104786545924968,Working,Secondary / secondary special,Married,House / apartment,Security staff,N
2,5008808.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,4.0,270000.0,52.32140290355038,8.353354278321937,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,Sales staff,N


In [24]:
# Splitting data into train test and validation set 
data_train1,data_test1,data_valid1 = data1.split_frame(ratios=[.7, .15])

In [25]:
data_train1.head()

ID,Gender,Own_car,Own_property,Work_phone,Phone,Email,Unemployed,Num_children,Num_family,Account_length,Total_income,Age,Years_employed,Income_type,Education_type,Family_status,Housing_type,Occupation_type,Target
5.0088e+06,1,1,1,1,0,0,0,0,2,15,427500,32.8686,12.4356,Working,Higher education,Civil marriage,Rented apartment,Other,Y
5.00881e+06,0,0,1,0,1,1,0,0,1,4,270000,52.3214,8.35335,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,Sales staff,N
5.00881e+06,0,0,1,0,0,0,1,0,1,20,283500,61.5043,0,Pensioner,Higher education,Separated,House / apartment,Other,N
5.00882e+06,1,1,1,1,1,1,0,0,2,5,270000,46.194,2.10545,Working,Higher education,Married,House / apartment,Accountants,N
5.00882e+06,0,1,0,0,0,0,0,0,2,25,130500,29.2107,3.01991,Working,Incomplete higher,Married,House / apartment,Accountants,Y
5.00883e+06,0,0,1,0,0,0,0,1,2,44,112500,30.0294,4.43541,Working,Secondary / secondary special,Single / not married,House / apartment,Other,N
5.00884e+06,1,1,1,0,0,0,0,3,5,24,270000,34.7413,3.18419,Working,Secondary / secondary special,Married,House / apartment,Laborers,N
5.00884e+06,1,0,1,0,0,0,0,1,3,39,405000,32.4223,5.51962,Commercial associate,Higher education,Married,House / apartment,Managers,N
5.00884e+06,1,1,1,0,1,0,0,0,2,43,112500,56.1326,12.1837,Commercial associate,Secondary / secondary special,Married,House / apartment,Drivers,N
5.00887e+06,0,0,1,0,0,0,0,1,3,24,211500,44.3869,19.4364,State servant,Secondary / secondary special,Civil marriage,House / apartment,Core staff,N


In [26]:
y1 = "Target"
x1 = data1.columns
x1.remove(y1)
x1

['ID',
 'Gender',
 'Own_car',
 'Own_property',
 'Work_phone',
 'Phone',
 'Email',
 'Unemployed',
 'Num_children',
 'Num_family',
 'Account_length',
 'Total_income',
 'Age',
 'Years_employed',
 'Income_type',
 'Education_type',
 'Family_status',
 'Housing_type',
 'Occupation_type']

In [27]:
aml1 = H2OAutoML(max_models = 10, seed = 10, verbosity="info", nfolds=0)

In [28]:
aml1.train(x = x1, y = y1, training_frame = data_train1, validation_frame=data_valid1)

AutoML progress: |
01:27:18.760: Project: AutoML_2_20221108_12718
01:27:18.760: Cross-validation disabled by user: no fold column nor nfolds > 1.
01:27:18.761: Setting stopping tolerance adaptively based on the training frame: 0.012133920952907253
01:27:18.761: Build control seed: 10
01:27:18.761: training frame: Frame key: AutoML_2_20221108_12718_training_py_12_sid_b8a4    cols: 20    rows: 6792  chunks: 1    size: 232145  checksum: 6118467675611229324
01:27:18.761: validation frame: Frame key: py_14_sid_b8a4    cols: 20    rows: 1468  chunks: 1    size: 56362  checksum: -3625328837734852563
01:27:18.761: leaderboard frame: Frame key: py_14_sid_b8a4    cols: 20    rows: 1468  chunks: 1    size: 56362  checksum: -3625328837734852563
01:27:18.761: blending frame: NULL
01:27:18.761: response column: Target
01:27:18.761: fold column: null
01:27:18.761: weights column: null
01:27:18.762: Loading execution steps: [{XGBoost : [def_2 (1g, 10w), def_1 (2g, 10w), def_3 (3g, 10w), grid_1 (4g, 90

Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_1_AutoML_2_20221108_12718


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    30                 30                          17157                  9            15           12.0667       36            43            40.6667

ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.09992722113153554
RMSE: 0.31611267157697986
LogLoss: 0.33244050654039475
Mean Per-Class Error: 0.2790978717973022
AUC: 0.843634865472851
AUCPR: 0.5028490537660107
Gini: 0.6872697309457021

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.19504256966250394
       N     Y     Error    Rate
-----  ----  ----  -------  --------------
N      5346  541   0.0919   (541.0/5887.0)
Y      422   483   0.4663   (422.0/905.0)
Total  5768  1024  0.1418   (963.0/6792.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.195043     0.500778  130
max f2                       0.14803      0.610301  200
max f0point5                 0.219492     0.518605  101
max accuracy                 0.256788     0.88192   66
max precision                0.437754     1         0
max recall                   0.0599162    1         356
max specificity              0.437754     1         0
max absolute_mcc             0.20453      0.420262  119
max min_per_class_accuracy   0.155835     0.758281  187
max mean_per_class_accuracy  0.14803      0.76441   200
max tns                      0.437754     5887      0
max fns                      0.437754     904       0
max fps                      0.0214292    5887      399
max tps                      0.0599162    905       356
max tnr                      0.437754     1         0
max fnr                      0.437754     0.998895  0
max fpr                      0.0214292    1         399
max tpr                      0.0599162    1         356

Gains/Lift Table: Avg response rate: 13.32 %, avg score: 13.36 %
group    cumulative_data_fraction    lower_threshold    lift       cumulative_lift    response_rate    score      cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ---------  -----------------  ---------------  ---------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0100118                   0.318605           6.4013     6.4013             0.852941         0.349066   0.852941                    0.349066            0.0640884       0.0640884                  540.13    540.13             0.0623897
2        0.0200236                   0.291213           5.40799    5.90465            0.720588         0.303719   0.786765                    0.326393            0.0541436       0.118232                   440.799   490.465            0.113306
3        0.0300353                   0.271822           4.52506    5.44478            0.602941         0.281221   0.72549                     0.311335            0.0453039       0.163536                   352.506   444.478            0.154023
4        0.0400471                   0.256868           4.19396    5.13208            0.558824         0.26403    0.683824                    0.299509            0.041989        0.205525                   319.396   413.208            0.190916
5        0.0500589                   0.247375           3.53175    4.81201            0.470588         0.251939   0.641176                    0.

In [29]:
lb1 = aml1.leaderboard

In [30]:
lb1.head()

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
GBM_1_AutoML_2_20221108_12718,0.575651,0.362843,0.141709,0.419533,0.323416,0.104598
XGBoost_2_AutoML_2_20221108_12718,0.573192,0.384739,0.136138,0.435552,0.333789,0.111415
GLM_1_AutoML_2_20221108_12718,0.566322,0.357181,0.172085,0.431988,0.319706,0.102212
XGBoost_1_AutoML_2_20221108_12718,0.561426,0.38214,0.134232,0.441308,0.331666,0.110002
GBM_4_AutoML_2_20221108_12718,0.558676,0.372962,0.13587,0.444033,0.327673,0.107369
XGBoost_3_AutoML_2_20221108_12718,0.55823,0.371555,0.140994,0.446339,0.326549,0.106634
XRT_1_AutoML_2_20221108_12718,0.556107,0.370918,0.135206,0.445138,0.327106,0.106998
GBM_2_AutoML_2_20221108_12718,0.551668,0.368164,0.132769,0.43928,0.325581,0.106003
DRF_1_AutoML_2_20221108_12718,0.550075,0.377588,0.144262,0.469784,0.329265,0.108415
GBM_3_AutoML_2_20221108_12718,0.543288,0.371096,0.126957,0.452583,0.326745,0.106762


In [53]:
# finding and storing the best model
best_model1 = h2o.get_model(aml1.leaderboard[2,'model_id'])

In [54]:
# printing the best model
best_model1.algo

'glm'

In [31]:
data_pred1=aml1.leader.predict(data_test1)

gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [32]:
data_pred1.head()

predict,N,Y
Y,0.768759,0.231241
Y,0.855301,0.144699
Y,0.785175,0.214825
Y,0.861151,0.138849
Y,0.866732,0.133268
N,0.92567,0.07433
Y,0.736602,0.263398
N,0.947114,0.052886
N,0.900045,0.0999554
Y,0.859703,0.140297


In [34]:
aml1.leader.model_performance(data_test1)

ModelMetricsBinomial: gbm
** Reported on test data. **

MSE: 0.12238083845173382
RMSE: 0.34982972779873045
LogLoss: 0.4099628549611307
Mean Per-Class Error: 0.4408183671868873
AUC: 0.5592737824484354
AUCPR: 0.15611151154793834
Gini: 0.1185475648968708

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.11186070340749044
       N    Y    Error    Rate
-----  ---  ---  -------  --------------
N      481  763  0.6133   (763.0/1244.0)
Y      55   150  0.2683   (55.0/205.0)
Total  536  913  0.5645   (818.0/1449.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value      idx
---------------------------  -----------  ---------  -----
max f1                       0.111861     0.268336   258
max f2                       0.0502217    0.461747   370
max f0point5                 0.157224     0.19807    166
max accuracy                 0.341367     0.857833   0
max precision                0.179483     0.18123    130
max recall                   0.0383163    1          383
max specificity              0.341367     0.999196   0
max absolute_mcc             0.111861     0.0854449  258
max min_per_class_accuracy   0.133199     0.541801   214
max mean_per_class_accuracy  0.111861     0.559182   258
max tns                      0.341367     1243       0
max fns                      0.341367     205        0
max fps                      0.0193279    1244       399
max tps                      0.0383163    205        383
max tnr                      0.341367     0.999196   0
max fnr                      0.341367     1          0
max fpr                      0.0193279    1          399
max tpr                      0.0383163    1          383

Gains/Lift Table: Avg response rate: 14.15 %, avg score: 13.49 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score      cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  ---------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.010352                    0.287034           0         0                  0                0.309162   0                           0.309162            0               0                          -100      -100               -0.0120579
2        0.0200138                   0.269562           1.51463   0.731203           0.214286         0.27669    0.103448                    0.293486            0.0146341       0.0146341                  51.4634   -26.8797           -0.00626618
3        0.0303658                   0.254569           0.47122   0.642572           0.0666667        0.262547   0.0909091                   0.282938            0.00487805      0.0195122                  -52.878   -35.7428           -0.0126421
4        0.0400276                   0.245421           0.504878  0.609336           0.0714286        0.250347   0.0862069                   0.275071            0.00487805      0.0243902                  -49.5122  -39.0664           -0.0182143
5        0.0503796                   0.23698            1.41366   0.774607           0.2              0.240655   0.109589                    0.267999            0.0146341       0.0390244                  41.3659   -22.5393           -0.0132264
6        0.100069                    0.212204           1.37439   1.07243            0.194444         0.225022   0.151724                    0.246659            0.0682927       0.107317                   37.439    7.24306            0.00844248
7        0.150449                    0.195572           1.35556   1.16724            0.191781         0.204162   0.165138                    0.232428            0.0682927       0.17561                    35.5563   16.7241            0.0293075
8     

##REGRESSION 

In [36]:
data2 = h2o.H2OFrame(df_copy)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [37]:
data2.describe()

Rows:9709
Cols:20

,ID,Gender,Own_car,Own_property,Work_phone,Phone,Email,Unemployed,Num_children,Num_family,Account_length,Total_income,Age,Years_employed,Income_type,Education_type,Family_status,Housing_type,Occupation_type,Target
type,int,int,int,int,int,int,int,int,int,int,int,real,real,real,enum,enum,enum,enum,enum,enum
mins,5008804.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,27000.0,20.504185575336933,0.0,,,,,,
mean,5076104.679060664,0.3487485837882377,0.36770007209805333,0.671541868369554,0.21742712946750437,0.2876712328767123,0.08754763621382222,0.1746832835513441,0.42280358430322246,2.1826140694201257,27.270058708414904,181228.19456174638,43.784093083598485,5.664730161452851,,,,,,
maxs,5150479.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,19.0,20.0,60.0,1575000.0,68.86383703977495,43.02073280081042,,,,,,
sigma,40802.696052772466,0.4765987878099639,0.4822039797226454,0.4696765995682179,0.4125167873991112,0.45270034532244624,0.2826504487639636,0.3797155310732071,0.7670189141552809,0.9329181887062356,16.648056893148983,99277.30509737751,11.625767724979015,6.3422412768887755,,,,,,
zeros,0,6323,6139,3189,7598,6916,8859,8013,6819,0,57,0,0,1696,,,,,,
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,5008804.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,2.0,15.0,427500.0,32.86857361889703,12.4355736257418,Working,Higher education,Civil marriage,Rented apartment,Other,Y
1,5008806.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,29.0,112500.0,58.79381506807121,3.104786545924968,Working,Secondary / secondary special,Married,House / apartment,Security staff,N
2,5008808.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,4.0,270000.0,52.32140290355038,8.353354278321937,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,Sales staff,N


Splitting data into train test and validation set 

In [38]:
data_train2,data_test2,data_valid2 = data2.split_frame(ratios=[.7, .15])

In [39]:
data_train2.head()

ID,Gender,Own_car,Own_property,Work_phone,Phone,Email,Unemployed,Num_children,Num_family,Account_length,Total_income,Age,Years_employed,Income_type,Education_type,Family_status,Housing_type,Occupation_type,Target
5.0088e+06,1,1,1,1,0,0,0,0,2,15,427500,32.8686,12.4356,Working,Higher education,Civil marriage,Rented apartment,Other,Y
5.00881e+06,1,1,1,0,0,0,0,0,2,29,112500,58.7938,3.10479,Working,Secondary / secondary special,Married,House / apartment,Security staff,N
5.00881e+06,0,0,1,0,1,1,0,0,1,4,270000,52.3214,8.35335,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,Sales staff,N
5.00881e+06,0,0,1,0,0,0,1,0,1,20,283500,61.5043,0,Pensioner,Higher education,Separated,House / apartment,Other,N
5.00882e+06,1,1,1,1,1,1,0,0,2,5,270000,46.194,2.10545,Working,Higher education,Married,House / apartment,Accountants,N
5.00882e+06,1,1,1,0,0,0,0,0,2,17,135000,48.6745,3.26906,Commercial associate,Secondary / secondary special,Married,House / apartment,Laborers,N
5.00883e+06,0,0,1,0,1,0,0,0,2,31,157500,27.4639,4.02199,Working,Secondary / secondary special,Married,House / apartment,Laborers,Y
5.00884e+06,1,0,1,0,0,0,0,1,3,39,405000,32.4223,5.51962,Commercial associate,Higher education,Married,House / apartment,Managers,N
5.00884e+06,1,1,1,0,1,0,0,0,2,43,112500,56.1326,12.1837,Commercial associate,Secondary / secondary special,Married,House / apartment,Drivers,N
5.00885e+06,0,1,1,0,0,0,0,2,4,39,135000,43.1522,8.68738,Working,Secondary / secondary special,Married,House / apartment,Laborers,N


In [40]:
y2 = "Total_income"
x2 = data.columns
x2.remove(y2)
x2

['ID',
 'Gender',
 'Own_car',
 'Own_property',
 'Work_phone',
 'Phone',
 'Email',
 'Unemployed',
 'Num_children',
 'Num_family',
 'Account_length',
 'Age',
 'Years_employed',
 'Income_type',
 'Education_type',
 'Family_status',
 'Housing_type',
 'Occupation_type',
 'Target']

In [41]:
aml2 = H2OAutoML(max_models = 10, seed = 10, verbosity="info", nfolds=0)

In [42]:
aml2.train(x = x2, y = y2, training_frame = data_train2, validation_frame=data_valid2)

AutoML progress: |
01:53:26.834: Project: AutoML_3_20221108_15326
01:53:26.834: Cross-validation disabled by user: no fold column nor nfolds > 1.
01:53:26.835: Setting stopping tolerance adaptively based on the training frame: 0.012113428212804355
01:53:26.835: Build control seed: 10
01:53:26.835: training frame: Frame key: AutoML_3_20221108_15326_training_py_21_sid_b8a4    cols: 20    rows: 6815  chunks: 1    size: 232932  checksum: -1641328085119881042
01:53:26.835: validation frame: Frame key: py_23_sid_b8a4    cols: 20    rows: 1424  chunks: 1    size: 54712  checksum: 6029893344071383551
01:53:26.835: leaderboard frame: Frame key: py_23_sid_b8a4    cols: 20    rows: 1424  chunks: 1    size: 54712  checksum: 6029893344071383551
01:53:26.835: blending frame: NULL
01:53:26.835: response column: Total_income
01:53:26.835: fold column: null
01:53:26.835: weights column: null
01:53:26.836: Loading execution steps: [{XGBoost : [def_2 (1g, 10w), def_1 (2g, 10w), def_3 (3g, 10w), grid_1 (4

,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
,35.0,35.0,20105.0,9.0,15.0,11.914286,36.0,43.0,40.8
,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance,validation_rmse,validation_mae,validation_deviance
,2022-11-08 01:53:29,0.018 sec,0.0,98471.0800812,68962.5998195,9696553612.3628674,102569.2340443,70147.2894918,10520447772.4353180
,2022-11-08 01:53:29,0.205 sec,5.0,91465.3578330,63671.2788472,8365911683.5268526,97978.8231539,66142.7177719,9599849786.6152401
,2022-11-08 01:53:29,0.418 sec,10.0,87796.1303729,60966.4870025,7708160508.4555235,95971.7324184,64631.6011880,9210573423.3894215
,2022-11-08 01:53:29,0.621 sec,15.0,85756.0079520,59550.7453584,7354092899.8617115,95157.3937543,64071.3029025,9054929586.1021805
,2022-11-08 01:53:30,0.800 sec,20.0,84362.8817801,58549.3483171,7117095822.2464504,94873.4077383,63818.6415003,9000963495.8785954
,2022-11-08 01:53:30,0.960 sec,25.0,83355.1530064,57792.9292175,6948081532.7191648,94827.0478895,63773.5829998,8992169011.4331570
,2022-11-08 01:53:30,1.126 sec,30.0,82508.8254300,57168.0936422,6807706273.8369932,94986.2229046,63906.9518507,9022382541.6787796
,2022-11-08 01:53:30,1.350 sec,35.0,81686.3500335,56644.3292679,6672659781.7995529,95114.0571298,64066.4789448,9046683863.6998844


In [43]:
lb2 = aml2.leaderboard

In [44]:
lb2.head()

model_id,rmse,mse,mae,rmsle,mean_residual_deviance
GBM_1_AutoML_3_20221108_15326,95114.1,9.04668e+09,64066.5,0.455016,9.04668e+09
GBM_3_AutoML_3_20221108_15326,96244.3,9.26296e+09,64705.1,0.457338,9.26296e+09
GBM_2_AutoML_3_20221108_15326,96461,9.30472e+09,64580.6,0.457398,9.30472e+09
XRT_1_AutoML_3_20221108_15326,97283.1,9.464e+09,66153.5,0.46896,9.464e+09
XGBoost_3_AutoML_3_20221108_15326,97343.2,9.47569e+09,65788,0.465796,9.47569e+09
GBM_4_AutoML_3_20221108_15326,97491.6,9.5046e+09,65492.3,0.462699,9.5046e+09
DRF_1_AutoML_3_20221108_15326,98382.4,9.6791e+09,66026.5,0.46888,9.6791e+09
GLM_1_AutoML_3_20221108_15326,102569,1.05204e+10,70147.3,0.502947,1.05204e+10
XGBoost_2_AutoML_3_20221108_15326,103115,1.06326e+10,69203.5,0.498492,1.06326e+10
XGBoost_1_AutoML_3_20221108_15326,104251,1.08682e+10,71126.6,0.508771,1.08682e+10


In [55]:
# finding and storing the best model
best_model2 = h2o.get_model(aml2.leaderboard[0,'model_id'])

In [56]:
# printing the best model
best_model2.algo

'gbm'

In [45]:
data_pred2=aml2.leader.predict(data_test2)

gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [46]:
data_pred2.head()

predict
154439
199242
258070
181942
171431
163011
167728
155240
167881
206183


In [47]:
aml2.leader.model_performance(data_test2)

ModelMetricsRegression: gbm
** Reported on test data. **

MSE: 8056347909.726555
RMSE: 89757.16077130869
MAE: 62219.4875280458
RMSLE: 0.44957592033783816
Mean Residual Deviance: 8056347909.726555

##ANSWERS

**1.** Is the relationship significant?

**Answer:** 

**Multinomial Classification-**
Yes, the 'Total_income', 'Years_employed' are significant variables.

**Binary Classification-**
Yes, the 'Total Income' are significant variables.

**Regression-**
Yes, the 'Total_income','Years_employed' are significant variables.

**2.** Are any model assumptions violated?

**Answer:** 

**Multinomial Classification-**
The best model is glm and there are 4 assumptions to validate which  are linearity, homoskedasticity (constant variance), normality and Independence.

**Binary Classification-**
The best model is glm and there are 3 assumptions to validate which  are linearity, homoskedasticity (constant variance), normality.

**Regression-**
For GBM model there are no assumtions to validate because it is a tree based model. Hence, it serves to be the best model.


**3.**  Is there any multicollinearity in the model?

**Answer:** Yes, there is multicollinearity in the Regression model

**4.** In the multivariate models are predictor variables independent of all the other predictor variables?

 **Answer:** 

 Yes, the predictor variables are independent of all the other predictor variables in the multivariate models.


**5.** In multivariate models rank the most significant predictor variables and exclude insignificant ones from the model.

**Answer:**
The top 3 significant predictor variables for the multivariate model are ranked as follows- 

  1. Years_employed	
  2. Unemployed	
  3. ID	

**6.** Does the model make sense?

**Answer:**

**Multinomial Classification-**
In this, the models used does make sense. The log loss vlaues is very close to 0 and the AUC value is 1 which is the score a perfect classifier should help.


**Binary Classification-**
The models used does make sense. The log loss vlaues is very close to 0 and the AUC value is 1 which is the score a perfect classifier should help.

**Regression-**
The model does not look like a optimal solution because we are trying to predict 'Income' based on the factors which is not the case as multicollinearity is found in the data. So, it does not make sense for Regression.  

**7.** Does regularization help?

**Answer:**

Yes, regularization helps in optimizing the results.

**8.** Which hyperparameters are important?

**Answer:**

A simple GBM model in Regression contains two categories of hyperparameters: boosting hyperparameters and tree-specific hyperparameters. The two main boosting hyperparameters include: Number of trees: The total number of trees in the sequence or ensemble. 

##REFRENCES

1. 6105_Airlines_GBM_AutoML.ipynb 
2. Python Data Science Handbook
3. Sckit learn offcial documentation
4. AmazonReviews.ipynb


Refered to the sample notebook (Abalone dataset) to undertsand expected assignment format. 

Used Sckitlearn tools to implement models. Used mlxtend for bias and variance. Used eli5 to predict and understand feature importance. 

Refered to Analytics Vidhya and Python Data science Handbook to study different models and concepts. 

Copyright 2022 *Jatin Madan*

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.